In [349]:
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')

In [350]:
import os
print(os.getcwd())

C:\Users\guswh\Desktop\data-analysis\dacon_housePrediction\src


In [351]:
train = pd.read_csv('../data/prep/train3_train.csv')
test = pd.read_csv('../data/prep/test3_valid.csv')
submission = pd.read_csv('../data/prep/submission_valid.csv')

In [352]:
train

,id,Gr.Liv.Area,Garage.Cars,Garage.Area,Total.Bsmt.SF,X1st.Flr.SF,Full.Bath,Year.Built,Garage.Yr.Blt_cal,Year.Remod.Add_cal,Year.Built_cal,comb.Built,total.Price.Index,target
0,1142,6.872128,1.098612,6.665684,6.872128,6.872128,0.693147,7.581720,3.295837,3.931826,3.931826,4.615121,12.016249,11.831386
1,1206,7.357556,1.098612,6.572283,7.357556,7.357556,1.098612,7.589842,3.555348,3.555348,3.555348,4.234107,12.691276,12.185875
2,228,7.482682,1.386294,6.790097,7.368340,7.482682,1.098612,7.580189,3.784190,3.988984,3.988984,4.672829,12.510173,11.608245
3,549,7.278629,1.098612,6.541030,7.224753,7.278629,1.098612,7.604396,1.791759,1.609438,1.791759,2.302585,13.073511,12.382129
4,325,7.284135,1.098612,6.018593,6.591674,6.591674,1.098612,7.603898,1.945910,1.945910,1.945910,2.564949,12.539143,12.149508
5,542,7.884200,1.098612,6.429719,6.961296,7.321850,1.098612,7.585281,3.784190,3.332205,3.784190,4.262680,12.766662,12.429220
6,1084,6.930495,0.693147,5.638355,6.930495,6.930495,0.693147,7.555382,4.025352,1.791759,4.624973,4.672829,11.922190,11.350418
7,33,6.985642,1.098612,6.184149,6.989335,6.985642,0.693147,7.592870,3.367296,3.367296,3.367296,4.043051,12.212616,11.805603
8,251,7.758333,1.386294,6.669498,7.120444,7.120444,1.098612,7.599401,2.772589,2.772589,2.772589,3.433987,13.194693,12.631344
9,744,7.346655,1.098612,6.084499,6.996681,7.014814,1.098612,7.604396,1.609438,1.609438,1.791759,2.302585,12.771753,12.392556


In [353]:
train = train.iloc[:, 1:]
test = test.iloc[:, 1:]

In [354]:
X = train.drop('target', axis = 1)
#y = np.log1p(train.target)
y = train.target
target = test[X.columns]

In [355]:
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from catboost import CatBoostRegressor, Pool
from ngboost import NGBRegressor
from sklearn.metrics import make_scorer
from sklearn.model_selection import KFold

In [356]:
def NMAE(true, pred) -> float:
    mae = np.mean(np.abs(true - pred))
    score = mae / np.mean(np.abs(true))
    return score

In [357]:
nmae_score = make_scorer(NMAE, greater_is_better=False)

In [358]:
kf = KFold(n_splits = 10, random_state = 42, shuffle = True)

In [359]:
cb_pred = np.zeros(target.shape[0])
cb_val = []
for n, (tr_idx, val_idx) in enumerate(kf.split(X, y)) :
    print(f'{n + 1} FOLD Training.....')
    tr_x, tr_y = X.iloc[tr_idx], y.iloc[tr_idx]
    val_x, val_y = X.iloc[val_idx], np.expm1(y.iloc[val_idx])
    
    tr_data = Pool(data = tr_x, label = tr_y)
    val_data = Pool(data = val_x, label = val_y)
    
    cb = CatBoostRegressor(depth = 4, random_state = 42, loss_function = 'MAE', n_estimators = 3000, learning_rate = 0.03, verbose = 0)
    cb.fit(tr_data, eval_set = val_data, early_stopping_rounds = 750, verbose = 1000)
    
    val_pred = np.expm1(cb.predict(val_x))
    val_nmae = NMAE(val_y, val_pred)
    cb_val.append(val_nmae)
    print(f'{n + 1} FOLD NMAE = {val_nmae}\n')
    
    target_data = Pool(data = target, label = None)
    fold_pred = cb.predict(target) / 10
    cb_pred += fold_pred
print(f'10FOLD Mean of NMAE = {np.mean(cb_val)} & std = {np.std(cb_val)}')

1 FOLD Training.....
0:	learn: 0.2898525	test: 188296.9299860	best: 188296.9299860 (0)	total: 1.95ms	remaining: 5.85s
Stopped by overfitting detector  (750 iterations wait)

bestTest = 188296.9053
bestIteration = 79

Shrink model to first 80 iterations.
1 FOLD NMAE = 0.12296045087915863

2 FOLD Training.....
0:	learn: 0.2933618	test: 187793.1953848	best: 187793.1953848 (0)	total: 1.47ms	remaining: 4.42s
Stopped by overfitting detector  (750 iterations wait)

bestTest = 187793.1438
bestIteration = 114

Shrink model to first 115 iterations.
2 FOLD NMAE = 0.10271094917754414

3 FOLD Training.....
0:	learn: 0.2919223	test: 191102.0003491	best: 191102.0003491 (0)	total: 2.46ms	remaining: 7.39s
1000:	learn: 0.0639458	test: 191101.9556203	best: 191101.9556006 (998)	total: 1.72s	remaining: 3.44s
2000:	learn: 0.0554226	test: 191101.9553510	best: 191101.9549071 (1786)	total: 3.06s	remaining: 1.52s
Stopped by overfitting detector  (750 iterations wait)

bestTest = 191101.9549
bestIteration = 1786

In [360]:
# ngboost
ngb_pred = np.zeros(target.shape[0])
ngb_val = []
for n, (tr_idx, val_idx) in enumerate(kf.split(X, y)) :
    print(f'{n + 1} FOLD Training.....')
    tr_x, tr_y = X.iloc[tr_idx], y.iloc[tr_idx]
    val_x, val_y = X.iloc[val_idx], np.expm1(y.iloc[val_idx])
    
    ngb = NGBRegressor(random_state = 42, n_estimators = 1000, verbose = 0, learning_rate = 0.03)
    ngb.fit(tr_x, tr_y, val_x, val_y, early_stopping_rounds = 300)
    
    val_pred = np.expm1(ngb.predict(val_x))
    val_nmae = NMAE(val_y, val_pred)
    ngb_val.append(val_nmae)
    print(f'{n + 1} FOLD NMAE = {val_nmae}\n')
    
    target_data = Pool(data = target, label = None)
    fold_pred = ngb.predict(target) / 10
    ngb_pred += fold_pred
print(f'10FOLD Mean of NMAE = {np.mean(ngb_val)} & std = {np.std(ngb_val)}')

1 FOLD Training.....
1 FOLD NMAE = 0.10703972988930205

2 FOLD Training.....
2 FOLD NMAE = 0.09577298036831172

3 FOLD Training.....
3 FOLD NMAE = 0.10966164765578715

4 FOLD Training.....
4 FOLD NMAE = 0.09206918289538168

5 FOLD Training.....
5 FOLD NMAE = 0.10087228620542593

6 FOLD Training.....
6 FOLD NMAE = 0.0877635392802523

7 FOLD Training.....
7 FOLD NMAE = 0.1017064348055324

8 FOLD Training.....
8 FOLD NMAE = 0.11370933334806144

9 FOLD Training.....
9 FOLD NMAE = 0.08784033346604157

10 FOLD Training.....
10 FOLD NMAE = 0.09541336623165836

10FOLD Mean of NMAE = 0.09918488341457546 & std = 0.008528018123104686


In [361]:
submission['target'] = ngb_pred

In [362]:
submission.to_csv("../out/ngb/ngb1_valid.csv", header = True, index = False)

In [363]:
submission['target'] = cb_pred

In [364]:
submission.to_csv("../out/cb/cb1_valid.csv", header = True, index = False)